In [18]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

## Importing test data 

In [19]:
test_data = pd.read_csv('option_test_wolabel.csv')

In [20]:
test_data.head()

,S,K,tau,r
0,431.618600,460,0.293651,0.03147
1,432.633296,420,0.182540,0.03147
2,432.633296,430,0.182540,0.03147
3,431.618600,415,0.293651,0.03147
4,434.772855,420,0.043651,0.03147


## Import and preprocessing train data

In [21]:
data = pd.read_csv('option_train.csv')
print(data.head())
data.dtypes

       Value           S      K       tau        r     BS
0  21.670404  431.623898  420.0  0.341270  0.03013  Under
1   0.125000  427.015526  465.0  0.166667  0.03126   Over
2  20.691244  427.762336  415.0  0.265873  0.03116  Under
3   1.035002  451.711658  460.0  0.063492  0.02972   Over
4  39.553020  446.718974  410.0  0.166667  0.02962  Under


Value    float64
S        float64
K        float64
tau      float64
r        float64
BS        object
dtype: object

In [22]:
data.describe()

,Value,S,K,tau,r
count,1678.000000,1679.000000,1678.000000,1679.000000,1680.000000
mean,15.068709,464.402535,438.241955,0.437519,0.030235
std,14.040023,973.652179,23.408989,7.057555,0.000557
min,0.125000,0.000000,375.000000,0.003968,0.029510
25%,2.255001,433.863864,420.000000,0.119048,0.029820
50%,11.190967,442.634081,440.000000,0.202381,0.030130
75%,25.747434,447.320414,455.000000,0.285714,0.030540
max,60.149367,40333.000000,500.000000,250.000000,0.031880


In [23]:
data.isna().sum()

Value    2
S        1
K        2
tau      1
r        0
BS       0
dtype: int64

In [24]:
data = data.drop(data[data.S >40000].index)
data = data.drop(data[data.tau > 200].index)
data = data.drop(data[data.tau>140].index)
data = data.drop(data[data.S<5].index)

In [25]:
data = data.dropna()

In [26]:
data.isna().sum()

Value    0
S        0
K        0
tau      0
r        0
BS       0
dtype: int64

In [27]:
X = data.iloc[:, 1:5]
y = data.iloc[: , 0]

In [28]:
X, y

(               S      K       tau        r
 0     431.623898  420.0  0.341270  0.03013
 1     427.015526  465.0  0.166667  0.03126
 2     427.762336  415.0  0.265873  0.03116
 3     451.711658  460.0  0.063492  0.02972
 4     446.718974  410.0  0.166667  0.02962
 ...          ...    ...       ...      ...
 1675  444.397163  475.0  0.273810  0.03034
 1676  438.453825  410.0  0.301587  0.02972
 1677  428.042219  390.0  0.170635  0.03188
 1678  439.081203  480.0  0.293651  0.02962
 1679  432.167692  465.0  0.218254  0.02993
 
 [1673 rows x 4 columns],
 0       21.670404
 1        0.125000
 2       20.691244
 3        1.035002
 4       39.553020
           ...    
 1675     1.470000
 1676    34.927317
 1677    41.127997
 1678     0.440003
 1679     0.410002
 Name: Value, Length: 1673, dtype: float64)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Regression 

### Multi-linear regression

In [30]:
model1= smf.ols('Value ~ S + K + tau + r', data = data).fit()

In [31]:
model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Value   R-squared:                       0.912
Model:                            OLS   Adj. R-squared:                  0.912
Method:                 Least Squares   F-statistic:                     4315.
Date:                Wed, 07 Jun 2023   Prob (F-statistic):               0.00
Time:                        17:05:20   Log-Likelihood:                -4762.6
No. Observations:                1673   AIC:                             9535.
Df Residuals:                    1668   BIC:                             9562.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -22.2504     11.244     -1.979      0.048     -44.304      -0.197
S              0.6216      0.016     39.725      0.000       0.591       0.652
K             -0.5903      0.005   -129.944      0.000      -0.599      -0.581
tau           31.6364      1.048     30.198      0.000      29.582      33.691
r            515.3516    207.201      2.487      0.013     108.949     921.754
==============================================================================
Omnibus:                      156.358   Durbin-Watson:                   2.080
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              202.106
Skew:                           0.851   Prob(JB):                     1.30e-44
Kurtosis:                       3.063   Cond. No.                     1.26e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.26e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Forward selection

In [32]:
data.head()

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420.0,0.341270,0.03013,Under
1,0.125000,427.015526,465.0,0.166667,0.03126,Over
2,20.691244,427.762336,415.0,0.265873,0.03116,Under
3,1.035002,451.711658,460.0,0.063492,0.02972,Over
4,39.553020,446.718974,410.0,0.166667,0.02962,Under


In [33]:
def forward_selection(X,y, hidden_layers):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    
    selected_features = []
    
    best_accuracy = 0
    print(X_train.columns)
    
    for k in range(1, len(X_train.columns) +1 ):
        
        best_feature = None
        best_feature_accuracy = 0
        
        
        for curr_feature in X_train.columns :
            print('curr_feature == >{}'.format(curr_feature))
            if curr_feature in selected_features:
                continue
                
                
            features = list(selected_features) + [curr_feature]

            print(features)
            print('***')

            X_train_subset = X_train[features]
            X_test_subset = X_test[features]

            clf = MLPClassifier(hidden_layer_sizes=hidden_layers, activation='relu', solver='adam', max_iter=500)

            clf.fit(X_train_subset, y_train)


            accuracy = clf.score(X_test_subset, y_test)
            
            print('Accuracy == {}'.format(accuracy))
            print('\n\n')
            print('*******************************************')

        
            if accuracy > best_feature_accuracy:
                best_feature_accuracy = accuracy
                best_feature = curr_feature
            
        
        selected_features.append(best_feature)


        if best_feature_accuracy > best_accuracy :
            best_accuracy = best_feature_accuracy

            
    print('best accuracy = {}'.format(best_accuracy)) 

### KNN Regressor

In [34]:
from sklearn.metrics import r2_score

knn_reg = KNeighborsRegressor(n_neighbors=5)

# Fitting the model to the training data
knn_reg.fit(X_train, y_train)

# Predicting the target values for the testing set
y_pred = knn_reg.predict(X_test)

# Calculating mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

r2 = r2_score(y_test, y_pred)
print(r2)

MSE: 5.166724001925618
0.9737069715501093


### Decision Tree Regressor

In [35]:
from sklearn import tree

clf = tree.DecisionTreeRegressor()
decisionTree_regression = clf.fit(X_train, y_train)
y_pred_decisionTree = decisionTree_regression.predict(X_test)
print("out-of-sample:", decisionTree_regression.score(X_test, y_test))

out-of-sample: 0.9911382769841408


In [36]:
adjR2_formula2 = 1 - ( 1-decisionTree_regression.score(X_test, y_test) ) * ( len(y_test) - 1 ) / ( len(y_test) -X_test.shape[1] - 1 )
adjR2_formula2

0.9910669552697274

### Random Forest Regressor

In [37]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

randomForest_regression = rf_model.fit(X_train, y_train)

randomForest_regression.score(X_test, y_test)

0.9953878131909926

## Classification

In [38]:
data = pd.read_csv('option_train.csv')
data = data.dropna()

In [39]:
dic = {'Under': 0, 'Over': 1}

In [40]:
df2 = data

In [41]:
df2

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420.0,0.341270,0.03013,Under
1,0.125000,427.015526,465.0,0.166667,0.03126,Over
2,20.691244,427.762336,415.0,0.265873,0.03116,Under
3,1.035002,451.711658,460.0,0.063492,0.02972,Over
4,39.553020,446.718974,410.0,0.166667,0.02962,Under
...,...,...,...,...,...,...
1675,1.470000,444.397163,475.0,0.273810,0.03034,Over
1676,34.927317,438.453825,410.0,0.301587,0.02972,Under
1677,41.127997,428.042219,390.0,0.170635,0.03188,Under
1678,0.440003,439.081203,480.0,0.293651,0.02962,Over


In [42]:
df2['BS'] = df2['BS'].map(dic)

In [43]:
df2

,Value,S,K,tau,r,BS
0,21.670404,431.623898,420.0,0.341270,0.03013,0
1,0.125000,427.015526,465.0,0.166667,0.03126,1
2,20.691244,427.762336,415.0,0.265873,0.03116,0
3,1.035002,451.711658,460.0,0.063492,0.02972,1
4,39.553020,446.718974,410.0,0.166667,0.02962,0
...,...,...,...,...,...,...
1675,1.470000,444.397163,475.0,0.273810,0.03034,1
1676,34.927317,438.453825,410.0,0.301587,0.02972,0
1677,41.127997,428.042219,390.0,0.170635,0.03188,0
1678,0.440003,439.081203,480.0,0.293651,0.02962,1


In [44]:
X = df2.iloc[:, 1:5]
y = df2.iloc[:, -1]

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y )

In [46]:
df2['tau'] = df2['tau'].astype(str).astype(float)

### Logistic Regression Classifier

In [47]:
logistic_model = smf.logit('BS ~ S + K + tau + r', data = df2).fit()

Optimization terminated successfully.
         Current function value: 0.211749
         Iterations 13


/Users/tsunder/miniforge3/envs/tfmetal/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


In [48]:
logistic_model.summary()

/Users/tsunder/miniforge3/envs/tfmetal/lib/python3.9/site-packages/statsmodels/discrete/discrete_model.py:1819: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-X))


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     BS   No. Observations:                 1677
Model:                          Logit   Df Residuals:                     1672
Method:                           MLE   Df Model:                            4
Date:                Wed, 07 Jun 2023   Pseudo R-squ.:                  0.6909
Time:                        17:05:20   Log-Likelihood:                -355.10
converged:                       True   LL-Null:                       -1148.8
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     41.2131     10.360      3.978      0.000      20.908      61.519
S             -0.2435      0.019    -12.789      0.000      -0.281      -0.206
K              0.2107      0.012     18.070      0.000       0.188       0.234
tau            0.0117      0.041      0.282      0.778      -0.069       0.093
r           -891.1145    194.977     -4.570      0.000   -1273.262    -508.967
==============================================================================
"""

In [49]:
prediction_log = logistic_model.predict()
arr = logistic_model.pred_table()
arr

array([[877.,  68.],
       [ 80., 652.]])

In [50]:
(arr[0][0] + arr[1][1]) / ((arr[0][0] + arr[1][1]) +(arr[0][1] + arr[1][0]))

0.9117471675611211

### KNN Classifier

In [51]:
knn_model = KNeighborsClassifier(n_neighbors=5)

knn_model.fit(X_train, y_train)

y_pred = knn_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)

Accuracy:  0.9226190476190477


### Decision Tree Classifier

In [52]:
dtc = DecisionTreeClassifier()

dtc.fit(X_train, y_train)

y_pred = dtc.predict(X_test)

print('Accuracy:', dtc.score(X_test, y_test))

Accuracy: 0.9186507936507936


### Random Forest Classifier

In [53]:
rf = RandomForestClassifier(n_estimators=50, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9563492063492064


## Testing Prediction - Final models

### Regression - Random Forest Regressor

In [54]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

randomForest_regression = rf_model.fit(X_train, y_train)

randomForest_regression.score(X_test, y_test)

0.8483571446862996

In [55]:
y_predict = rf_model.predict(test_data.values)

/Users/tsunder/miniforge3/envs/tfmetal/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [56]:
y_predict

array([1.  , 0.  , 0.31, ..., 1.  , 0.  , 0.28])

In [57]:
dfpred = pd.DataFrame(y_predict)

In [58]:
dfpred.to_csv("Regression_results.csv")

### Classification - Random Forest Classifier

In [59]:
rf = RandomForestClassifier(n_estimators=230, random_state=42)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9543650793650794


In [60]:
y_test_pred = rf.predict(test_data)

In [61]:
dfpred = pd.DataFrame({'BS' : y_test_pred})

In [62]:
dfpred.to_csv('Classification_results.csv')